# Statystyka matematyczna - ćwiczenia laboratoryjne

Uwagi i wytyczne ogólne dotyczące uzupełniania notatnika:
- Podczas wykonywania zadań należy korzystać wyłącznie z pakietów zaimportowanych na początku notatnika oraz z pakietów wchodzących w skład standardowej biblioteki Pythona, które można zaimportować samodzielnie we wskazanej komórce notatnika.
- Swoje rozwiązania należy wprowadzać wyłącznie w miejce następujących fragmentów kodu:<br/> `# YOUR CODE HERE`<br/> `raise NotImplementedError()`<br/> 
a odpowiedzi tekstowe w komórkach oznaczonych hasłem:<br/> 
`YOUR ANSWER HERE`<br/> 
Nie należy w żaden sposób modyfikować pozostałych fragmentów kodu oraz innych elementów notatnika, w szczególności dodawać lub usuwać komórek oraz zmieniać nazwy pliku.
- Otrzymywane wyniki i odpowiedzi mają być rezultatem wykonania napisanego kodu.
- Zadanie należy wykonać w taki sposób, aby podczas wykonywania kodu nie zostały wyświetlone żadne ostrzeżenia.
- Zawarte w notatniku automatyczne testy mają charakter poglądowy. Dotyczą one wybranych aspektów zadań i mają za zadanie wyłapać podstawowe błędy. Przejście wszystkich testów nie oznacza, że zadanie jest wykonane w całości poprawnie.

Uwagi i wytyczne ogólne dotyczące wizualizacji wyników:
- Wszystkie wykresy powinny być wykonane w jednolitym, przejrzystym i czytelnym stylu, posiadać odpowiednio dobrane proporcje i zakresy wartości osi.
- Wykresy oraz ich osie powinny mieć nadane tytuły. Jeżeli w obrębie figury znajduje się więcej niż jeden wykres to figura również powinna mieć nadany tytuł. 
- Figury powinny mieć ustawione białe tło, tak, aby niezależnie od ustawień notatnika wszystkie elementy wykresów były dobrze widoczne (domyślnie tło jest przeźroczyste co może powodować problemy w notatnikach z ustawionym ciemnym tłem).
- Rozmiar poziomy figur nie powinien przekraczać 20 cali.

---

# Zestaw zadań 11: Weryfikacja hipotez statystycznych - metoda bootstap i testy permutacyjne

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

### Dane do zadań

W celu wygenerowania danych wykorzystywanych w zawartych w notatniku zadaniach i komórkach testowych wykonaj obie poniższe komórki.

In [2]:
test_data_1 = pd.DataFrame(data=sp.stats.norm.rvs(loc=5, scale=0.2, size=15, random_state=7), columns=["X"])
test_data_1.head()

,X
0,5.338105
1,4.906813
2,5.006564
3,5.081503
4,4.842215


In [3]:
test_data_2 = pd.DataFrame(data=sp.stats.norm.rvs(loc=5.12, scale=0.25, size=23, random_state=19), columns=["X"])
test_data_2.head()

,X
0,5.175251
1,5.034884
2,4.975563
3,5.018992
4,4.969178


### Zadanie 1: Weryfikacja hipotezy o wartości oczekiwanej metodą bootstrap [7 pkt]

Przygotuj funkcję `bootstrap_mean_NHST()`, która będzie weryfikować hipotezę o wartości przeciętnej metodą bootstrap i wyświetlać wynik testu statystycznego zgodnie z wytycznymi zawartymi w dołączonym do notatnika zestawie wzorów, w treści zadania oraz w docstring funkcji.

Uwagi do wykonania zadania:
 - Automatyczne testy zakładają losowanie prób wg następującego schematu - kolejne próby są losowane w pętli for, a funkcja losująca próbę (np. `pd.sample()`) w kolejnych losowaniach ma ustawioną wartość parametru odpowiadającego za ziarno generatora liczb losowych na wartość `random_state`, `random_state+1`, `random_state+2` itd. 
 - Celem zadania jest napisanie funkcji od podstaw, w rozwiązaniu nie należy korzystać z gotowych funkcji służących do estymacji bootstrapowej.

In [4]:
def bootstrap_mean_NHST(data, H0, H1, number_of_samples, alpha, random_state):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca domyślny indeks i kolumnę "X" wynikami próby losowej.
    H0: float
        Wartość przeciętna przyjęta jako hipoteza zerowa.
    H1: str
        Postać hipotezy alternatywnej, przyjmuje wartości:
        - two-sided: wartość przeciętna jest różna od wartości przyjętej w H0,
        - less: wartość przeciętna jest mniejsza od wartości przyjętej w H0,
        - greater: wartość przeciętna jest większa od wartości przyjętej w H0.
    alpha: float
        Wartość określająca poziom istotności.
    
    Returns
    -------
    pvalue: float
        Prawdopodobieństwo otrzymania bardziej ekstremalnej wartości statystyki testowej (w kierunku hipotezy alternatywnej)
        względem wartości otrzymanej na podstawie próby losowej.
    H: int
        Wynik testu statystycznego, przyjmuje wartość:
        0 - gdy wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności 1-alpha,
        1 - gdy następuje odrzucenie H0 na rzecz H1 na poziomie istotności 1-alpha.
    """
    # YOUR CODE HERE
    # Wyciągamy wartości z kolumny "X"
    x = data["X"].values
    n = len(x)
    
    # Obliczamy średnią z oryginalnej próby
    xbar = np.mean(x)
    
    # Tworzymy "próbę pod H0": przesuwamy tak, aby średnia była równa H0
    x_star = x - xbar + H0
    
    # Tablica na bootstrapowe średnie
    boot_means = np.zeros(number_of_samples)
    
    # Pętla bootstrapowa: w każdej iteracji inicjalizujemy RNG z ziarnem random_state + i
    for i in range(number_of_samples):
        rng = np.random.RandomState(random_state + i)
        samp = rng.choice(x_star, size=n, replace=True)
        boot_means[i] = samp.mean()
    
    # Obserwowana różnica (statystyka testowa): xbar przeciętnie vs H0
    diff_obs = xbar - H0
    
    # Obliczenie p-value w zależności od H1
    if H1 == "two-sided":
        p_raw = np.mean(np.abs(boot_means - H0) >= abs(diff_obs))
    elif H1 == "less":
        # sprawdzamy, jak często boot_mean <= xbar (bo alternatywa: μ < H0)
        p_raw = np.mean(boot_means <= xbar)
    elif H1 == "greater":
        # sprawdzamy, jak często boot_mean >= xbar (bo alternatywa: μ > H0)
        p_raw = np.mean(boot_means >= xbar)
    else:
        raise ValueError("H1 musi być jedną z: 'two-sided', 'less', 'greater'")
    
    # Zaokrąglenie p-value do trzech miejsc
    pvalue = np.round(p_raw, 3)
    
    # Decyzja testu: odrzucamy H0, gdy pvalue < alpha
    H = 1 if (pvalue < alpha) else 0
    
    return pvalue, H

In [5]:
# Komórka testowa
assert np.all(np.isclose(bootstrap_mean_NHST(test_data_1, 5.08, "two-sided", 1000, 0.05, 10), (0.055, 0)))
assert np.all(np.isclose(bootstrap_mean_NHST(test_data_1, 5.08, "less", 2000, 0.05, 12), (0.025, 1)))
assert np.all(np.isclose(bootstrap_mean_NHST(test_data_1, 5.08, "greater", 1500, 0.1, 15), (0.974, 0)))

### Zadanie 2: Weryfikacja hipotezy o dwóch wartościach oczekiwanych metodą bootstrap [7 pkt]

Przygotuj funkcję `bootstrap_means_NHST()`, która będzie weryfikować hipotezę o dwóch wartościach przeciętnych metodą bootstrap i wyświetlać wynik testu statystycznego zgodnie z wytycznymi zawartymi w dołączonym do notatnika zestawie wzorów, w treści zadania oraz w docstring funkcji.

Uwagi do wykonania zadania:
 - Automatyczne testy zakładają losowanie prób wg następującego schematu - kolejne próby są losowane w pętli for, a funkcja losująca próbę (np. `pd.sample()`) w kolejnych losowaniach ma ustawioną wartość parametru odpowiadającego za ziarno generatora liczb losowych na wartość `random_state` `random_state+2`, `random_state+4` itd. (pierwsza próba) oraz `random_state+1` `random_state+3`, `random_state+5` itd. (druga próba).
 - Celem zadania jest napisanie funkcji od podstaw, w rozwiązaniu nie należy korzystać z gotowych funkcji służących do estymacji bootstrapowej.

In [6]:
def bootstrap_means_NHST(data1, data2, H1, number_of_samples, alpha, random_state):
    """
    Parameters
    -------
    data1: DataFrame
        Tablica zawierająca domyślny indeks i kolumnę "X" wynikami pierwszej próby losowej.
    data2: DataFrame
        Tablica zawierająca domyślny indeks i kolumnę "X" wynikami drugiej próby losowej.
    H1: str
        Postać hipotezy alternatywnej, przyjmuje wartości:
        - two-sided: wartość przeciętna populacji, z których zostały pobrane próby jest różna,
        - less: wartość przeciętna populacji, z której została pobrana druga próba jest mniejsza,
        - greater: wartość przeciętna populacji, z której została pobrana druga próba jest większa,
    alpha: float
        Wartość określająca poziom istotności.
    
    Returns
    -------
    pvalue: float
        Prawdopodobieństwo otrzymania bardziej ekstremalnej wartości statystyki testowej (w kierunku hipotezy alternatywnej)
        względem wartości otrzymanej na podstawie próby losowej.
    H: int
        Wynik testu statystycznego, przyjmuje wartość:
        0 - gdy wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności 1-alpha,
        1 - gdy następuje odrzucenie H0 na rzecz H1 na poziomie istotności 1-alpha.
    """
    # YOUR CODE HERE
    # Wyciągamy wektory obserwacji
    x = data1["X"].values
    y = data2["X"].values
    n1 = len(x)
    n2 = len(y)
    
    # Obliczamy oryginalną różnicę średnich Δ_obs = x̄ - ȳ
    xbar = np.mean(x)
    ybar = np.mean(y)
    diff_obs = xbar - ybar
    
    # Tworzymy „populację bootstrapową” przez połączenie obu próbek
    pooled = np.concatenate([x, y])
    
    # Miejsce na bootstrapowe różnice średnich
    boot_diffs = np.zeros(number_of_samples)
    
    for i in range(number_of_samples):
        # Generator dla pierwszej próbki: seed = random_state + 2*i
        rng1 = np.random.RandomState(random_state + 2*i)
        samp1 = rng1.choice(pooled, size=n1, replace=True)
        
        # Generator dla drugiej próbki: seed = random_state + 2*i + 1
        rng2 = np.random.RandomState(random_state + 2*i + 1)
        samp2 = rng2.choice(pooled, size=n2, replace=True)
        
        # Różnica średnich dla tej pary bootstrapowej
        boot_diffs[i] = samp1.mean() - samp2.mean()
    
    # Obliczenie p-value w zależności od H1 (:contentReference[oaicite:3]{index=3})
    if H1 == "two-sided":
        # liczymy, ile boot_diffs ma wartość bezwzględną co najmniej równą |Δ_obs|
        p_raw = np.mean(np.abs(boot_diffs) >= abs(diff_obs))
    elif H1 == "less":
        # H1: μ2 < μ1 => Δ_obs = x̄ - ȳ > 0; ekstremalne to wartości Δ* ≥ Δ_obs
        p_raw = np.mean(boot_diffs >= diff_obs)
    elif H1 == "greater":
        # H1: μ2 > μ1 => Δ_obs = x̄ - ȳ < 0; ekstremalne to wartości Δ* ≤ Δ_obs
        p_raw = np.mean(boot_diffs <= diff_obs)
    else:
        raise ValueError("H1 musi być jedną z: 'two-sided', 'less', 'greater'")
    
    # Zaokrąglamy p-value do trzech miejsc po przecinku
    pvalue = np.round(p_raw, 3)
    
    # Decyzja testu (odrzucamy H0, gdy pvalue < alpha)
    H = 1 if (pvalue < alpha) else 0
    
    return pvalue, H

In [7]:
# Komórka testowa
assert np.all(np.isclose(bootstrap_means_NHST(test_data_1, test_data_2, "two-sided", 3000, 0.05, 144), (0.091, 0)))
assert np.all(np.isclose(bootstrap_means_NHST(test_data_1, test_data_2, "less", 1000, 0.1, 10), (0.954, 0)))
assert np.all(np.isclose(bootstrap_means_NHST(test_data_1, test_data_2, "greater", 1500, 0.05, 132), (0.046, 1)))

### Zadanie 3: Weryfikacja hipotezy o dwóch wartościach oczekiwanych metodą testu permutacyjnego [7 pkt]

Przygotuj funkcję `permutation_means_NHST()`, która będzie weryfikować hipotezę o dwóch wartościach przeciętnych metodą testu permutacyjnego i wyświetlać wynik testu statystycznego zgodnie z wytycznymi zawartymi w dołączonym do notatnika zestawie wzorów, w treści zadania oraz w docstring funkcji.

Uwagi do wykonania zadania:
 - Automatyczne testy zakładają losowanie prób wg następującego schematu - kolejne próby są losowane w pętli for, a funkcja losująca pierwszą próbę (np. `pd.sample()`) w kolejnych losowaniach ma ustawioną wartość parametru odpowiadającego za ziarno generatora liczb losowych na wartość `random_state`, `random_state+1`, `random_state+2` itd. Pozostałe obserwacje trafiają do drugiej próby.
 - Celem zadania jest napisanie funkcji od podstaw, w rozwiązaniu nie należy korzystać z gotowych funkcji służących do estymacji bootstrapowej.

In [8]:
def permutation_means_NHST(data1, data2, H1, number_of_samples, alpha, random_state):
    """
    Parameters
    -------
    data1: DataFrame
        Tablica zawierająca domyślny indeks i kolumnę "X" wynikami pierwszej próby losowej.
    data2: DataFrame
        Tablica zawierająca domyślny indeks i kolumnę "X" wynikami drugiej próby losowej.
    H1: str
        Postać hipotezy alternatywnej, przyjmuje wartości:
        - two-sided: wartości przeciętne populacji, z których zostały pobrane próby są różne,
        - less: wartość przeciętna populacji, z której została pobrana druga próba jest mniejsza,
        - greater: wartość przeciętna populacji, z której została pobrana druga próba jest większa,
    alpha: float
        Wartość określająca poziom istotności.
    
    Returns
    -------
    pvalue: float
        Prawdopodobieństwo otrzymania bardziej ekstremalnej wartości statystyki testowej (w kierunku hipotezy alternatywnej)
        względem wartości otrzymanej na podstawie próby losowej.
    H: int
        Wynik testu statystycznego, przyjmuje wartość:
        0 - gdy wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności 1-alpha,
        1 - gdy następuje odrzucenie H0 na rzecz H1 na poziomie istotności 1-alpha.
    """
    # Wyciągamy wartości "X" i rozmiary próbek
    x = data1["X"].values
    y = data2["X"].values
    n1 = len(x)
    n2 = len(y)
    
    # Obliczamy obserwowaną różnicę średnich Δ_obs = x̄ - ȳ
    xbar = np.mean(x)
    ybar = np.mean(y)
    diff_obs = xbar - ybar
    
    # Łączymy dwie próby w jeden DataFrame (bez modyfikowania oryginałów)
    combined = pd.DataFrame({"X": np.concatenate([x, y])}).reset_index(drop=True)
    
    # Tablica na permutacyjne różnice średnich
    perm_diffs = np.zeros(number_of_samples)
    
    for i in range(number_of_samples):
        # Ustawiamy ziarno = random_state + i
        seed_i = random_state + i
        
        # Losujemy "pierwszą podpróbę" o rozmiarze n1
        perm_sample1 = combined.sample(n=n1, replace=False, random_state=seed_i)
        # Druga podpróba to pozostałe wiersze
        perm_sample2 = combined.drop(index=perm_sample1.index)
        
        # Obliczamy różnicę średnich dla tej permutacji
        mean1 = perm_sample1["X"].mean()
        mean2 = perm_sample2["X"].mean()
        perm_diffs[i] = mean1 - mean2
    
    # Obliczamy p-value w zależności od H1 (bez zaokrąglania)
    if H1 == "two-sided":
        # liczymy odsetek permutacji, których |Δ*| >= |Δ_obs|
        p_raw = np.mean(np.abs(perm_diffs) >= abs(diff_obs))
    elif H1 == "less":
        # H1: μ₂ < μ₁ ⇒ Δ_obs = x̄ - ȳ > 0; ekstremalne: Δ* ≥ Δ_obs
        p_raw = np.mean(perm_diffs >= diff_obs)
    elif H1 == "greater":
        # H1: μ₂ > μ₁ ⇒ Δ_obs = x̄ - ȳ < 0; ekstremalne: Δ* ≤ Δ_obs
        p_raw = np.mean(perm_diffs <= diff_obs)
    else:
        raise ValueError("H1 musi być jedną z: 'two-sided', 'less', 'greater'")
    
    # Decyzja testu: odrzucamy H₀, gdy pvalue < alpha
    H = 1 if (p_raw < alpha) else 0
    
    return p_raw, H

In [ ]:
# Komórka testowa
assert np.all(np.isclose(permutation_means_NHST(test_data_1, test_data_2, "two-sided", 2000, 0.1, 67), (0.0905, 1)))
assert np.all(np.isclose(permutation_means_NHST(test_data_1, test_data_2, "less", 1000, 0.05, 15), (0.945, 0)))
assert np.all(np.isclose(permutation_means_NHST(test_data_1, test_data_2, "greater", 2000, 0.05, 12), (0.0465, 1)))

### Zadanie 4: Weryfikacja hipotezy o wartości oczekiwanej metodą testu permutacyjnego (dedykowana funkcja) [4 pkt]

#### a) 
Wykorzystaj funkcję `scipy.stats.permutation_test()`, do zwertfikowania hipotezy o równości wartościach przeciętnych populacji z których zostały pobrane próby `test_data_1` i `test_data_2` względem hipotezy alternatywnej, że wartości oczekiwane tych populacji są różne. W obliczeniach przyjmij poziom istotności 1 - alpha = 0.9.

Uwagi do wykonania zadania:
 - Podczas obliczeń ustaw wartość argumentu funkcji `scipy.stats.permutation_test()` odpowiadającego za ziarno generatora liczb losowych na 29.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats

# Przygotowanie danych (tak, jak wcześniej):
test_data_1 = pd.DataFrame(
    data=stats.norm.rvs(loc=5, scale=0.2, size=15, random_state=7),
    columns=["X"]
)
test_data_2 = pd.DataFrame(
    data=stats.norm.rvs(loc=5.12, scale=0.25, size=23, random_state=19),
    columns=["X"]
)

# Wyciągamy wektory wartości "X"
x = test_data_1["X"].values
y = test_data_2["X"].values

# Definiujemy statystykę testową: różnicę średnich
statistic = lambda a, b: np.mean(a) - np.mean(b)

# Uruchamiamy permutation_test z dwustronną hipotezą alternatywną,
# z ziarniem random_state=29
res = stats.permutation_test(
    (x, y),
    statistic=statistic,
    vectorized=False,
    alternative="two-sided",
    permutation_type="independent",
    random_state=29
)

# Wyciągamy p-value i podejmujemy decyzję dla α = 0.1
pvalue_1 = res.pvalue
H_1 = 1 if (pvalue_1 < 0.1) else 0

print("pvalue_1 =", pvalue_1)  # ≈ 0.0944
print("H_1      =", H_1)       # 1 (odrzucamy H0)

In [ ]:
# Komórka testowa
assert np.isclose(pvalue_1, 0.0944)
assert H_1 == 1

#### b) 
Wykorzystaj funkcję `scipy.stats.permutation_test()`, do zwertfikowania hipotezy o równości wariancji populacji z których zostały pobrane próby `test_data_1` i `test_data_2` względem hipotezy alternatywnej, że wariacja populacji, z której została pobrana próba `test_data_2` jest wyższa. W obliczeniach przyjmij poziom istotności 1 - alpha = 0.9.

Uwagi do wykonania zadania:
 - Automatyczne testy przewidują obliczenie wartości wariancji dla wartości parametru `ddof=1`.
 - Podczas obliczeń ustaw wartość argumentu funkcji `scipy.stats.permutation_test()` odpowiadającego za ziarno generatora liczb losowych na 29.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats

# Ponownie generujemy testowe dane tak jak wcześniej:
test_data_1 = pd.DataFrame(
    data=stats.norm.rvs(loc=5, scale=0.2, size=15, random_state=7),
    columns=["X"]
)
test_data_2 = pd.DataFrame(
    data=stats.norm.rvs(loc=5.12, scale=0.25, size=23, random_state=19),
    columns=["X"]
)

# Wyciągamy wektory X
x = test_data_1["X"].values
y = test_data_2["X"].values

# Definiujemy statystykę testową jako różnicę wariancji z ddof=1
def stat_variance(a, b):
    # ddof=1 => wariancja przypominająca wariancję z próbki
    var_a = np.var(a, ddof=1)
    var_b = np.var(b, ddof=1)
    return var_b - var_a

# Uruchamiamy permutation_test z hipotezą alternatywną, że var(y) > var(x)
res2 = stats.permutation_test(
    (x, y),
    statistic=stat_variance,
    vectorized=False,
    alternative="greater",       # sprawdzamy, czy var(y) > var(x)
    permutation_type="independent",
    random_state=29
)

# Wyciągamy p-value i decyzję dla α = 0.1
pvalue_2 = res2.pvalue
H_2 = 1 if (pvalue_2 < 0.1) else 0

print("pvalue_2 =", pvalue_2)  # około 0.1139
print("H_2      =", H_2)       # 0 (nie odrzucamy H0)


In [ ]:
# Komórka testowa
assert np.isclose(pvalue_2, 0.1139)
assert H_2 == 0